In [1]:
import os
import optuna
import numpy as np
import pandas as pd
from time import sleep, time
from scipy.special import logit
from scipy.stats import beta
from datetime import datetime

In [2]:
# Generate and save inferred model if does not exist
if not os.path.isfile(f"bayesian_model/glmm.pickle"):
    os.system("python3 infer_bayesian_model.py")

In [3]:
def run_command_gnome(command, title="test", keep_window=False, test=False):
    if keep_window:
        command_suffix = "; $SHELL"
    else:
        command_suffix = ""
        
    if test:
        return f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """
    else:
        os.system(f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """)

## Power simulations

In [4]:
def collate_results(sim_name, test="bht", crit_val=0.05):
    try:
        df = pd.read_csv(f"data/simulations/{sim_name}.csv")
        success = (df[f"{test}_pval"] < crit_val).sum()
        fail = (df[f"{test}_pval"] >= crit_val).sum()
        return success, fail
    except FileNotFoundError:
        return 0, 0

In [5]:
def beta_ci(n_success, n_fail, level=0.95, l_perc=None, u_perc=None):
    a = n_success + 1
    b = n_fail+1
    if l_perc==None:
        l_perc = (1-level)/2
    if u_perc==None:
        u_perc = 1-(1-level)/2
    return beta.ppf(l_perc, a, b), beta.ppf(u_perc, a, b)

In [6]:
def generate_sim_settings(p_diff, n_raters, scores_per_r, total_scores, trials_per_sim=100):
    os.system(f"""
        python3 generate_sim_settings.py --p_diff "{p_diff}" --n_raters "{n_raters}" \
        --scores_per_r {scores_per_r} --total_scores {total_scores} --n_sims 1 \
        --trials_per_sim {trials_per_sim} --sim_name {sim_name}
    """)
    return()

In [7]:
def transfer_log(sim_name, power_id, sim_id):
    df = pd.read_csv(f"data/simulations/{sim_name}.csv")
    df.insert(0, "power_id", power_id)
    df["sim_id"] = sim_id
    
    # Overwrite file if start of a new power analysis
    if sim_id == power_id == 0:
        df.to_csv(f"data/simulations/power_{sim_name}.csv", index=False)
    else:
        log = pd.read_csv(f"data/simulations/power_{sim_name}.csv")
        pd.concat([log, df]).to_csv(f"data/simulations/power_{sim_name}.csv", index=False)
        
    return

In [8]:
timeout=75*60

def objective(n_raters, seed):
        generate_sim_settings(p_diff, n_raters, scores_per_r, total_scores, trials_per_sim=trials_per_sim)

        # Run simulations
        setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
        process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
        last_ran=[0]*N_PROCESSES
        
        while sum(setting_files)>0:
            for i in range(N_PROCESSES):
                # Intial pause to let pymc compile
                if last_ran[i] == 0:
                    sleep(5)
                    
                if setting_files[i] and not process_active[i]:
                    source = "source ~/anaconda3/etc/profile.d/conda.sh\n"
                    conda = "conda activate bayes_topic_measures\n"
                    c1 = f"""python3 perform_sig_test.py --trials_per_sim {trials_per_sim} --process {i} """
                    c2 = f""" --sim_name {sim_name} --optimal_alloc {OA} --n_runs {n_runs} --seed {seed}"""
#                     run_command_gnome(source+conda+c1+c2, title=f"p{i}", keep_window=False) # Laptop
                    run_command_gnome(c1+c2, title=f"p{i}", keep_window=False) # Desktop
                    last_ran[i] = time()
                
                # Removing process running file if timed out
                if time() - last_ran[i] > timeout:
                    os.system(f"rm data/{sim_name}/process_{i}_running")
                    last_ran[i] = time()
                    
            sleep(10)
            
#             # Calculating early termination
#             n_success, n_fail = collate_results(sim_name)
#             lower_ci, upper_ci = beta_ci(n_success, n_fail, level=CI_LEVEL)
            
#             if 0.9 < lower_ci or upper_ci < 0.9:
#                 # Wait for all processes to complete
#                 while sum(process_active) > 0:
#                     sleep(1)
#                     for i in range(N_PROCESSES):
#                         if time() - last_ran[i] > timeout:
#                             os.system(f"rm data/{sim_name}/process_{i}_running")
#                             last_ran[i] = time()
                            
#                     process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
                
#                 # Recalculate after waiting
#                 n_success, n_fail = collate_results(sim_name)
#                 lower_ci, upper_ci = beta_ci(n_success, n_fail, level=CI_LEVEL)
#                 if lower_ci < 0.9 < upper_ci:
#                     continue
                
#                 transfer_log(sim_name, power_id, sim_id[0])
#                 sim_id[0] += 1
#                 return abs(logit(0.9) - logit((n_success+1)/(n_success+n_fail+2)))
            
            # Refresh setting and process files
            process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
            setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
        
        transfer_log(sim_name, power_id, sim_id[0])
        sim_id[0] += 1
        return

In [9]:
# Settings for hoyle's significance testing
hoyle_total_scores = 50*26
hoyle_p_diff = 0.055
hoyle_scores_per_r = 38

In [10]:
# Simulation settings
sim_prefix = "power_3_"
N_PROCESSES = 6
trials_per_sim = 400
n_runs=30
seed=42

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
# CI_LEVEL = 0.91

In [11]:
# Simulation settings
sim_prefix = "power_4_"
N_PROCESSES = 6
trials_per_sim = 1100
n_runs=30
seed=442

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
# CI_LEVEL = 0.91

## Uniform Sampling

In [74]:
sim_name = sim_prefix+"045"
OA = False
sim_id = [0]

power_id, p_diff = 0, 0.045

for n_raters in [80, 110, 140, 170, 200]:
    print(datetime.now())
    objective(n_raters, seed)

2022-11-30 15:44:41.813775

    p_diff=0.045, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-11-30 19:00:42.435073

    p_diff=0.045, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-11-30 22:17:22.836378

    p_diff=0.045, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-01 02:08:35.229415

    p_diff=0.045, <class 'float'>
    n_raters=170, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-01 06:10:29.566847

    p_diff=0.045, <class 'float'>
    n_raters=200, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [76]:
sim_name = sim_prefix+"055"
OA = False
sim_id = [0]

power_id, p_diff = 0, 0.055

for n_raters in [40, 60, 80, 110, 140]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-01 22:43:09.636577

    p_diff=0.055, <class 'float'>
    n_raters=40, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-02 02:45:53.056587

    p_diff=0.055, <class 'float'>
    n_raters=60, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-02 06:54:36.615632

    p_diff=0.055, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-02 10:45:33.939008

    p_diff=0.055, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-02 14:38:56.837420

    p_diff=0.055, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [ ]:
sim_name = sim_prefix+"070"
OA = False
sim_id = [0]

power_id, p_diff = 0, 0.070

for n_raters in [40, 50, 60, 70, 80]:
    print(datetime.now())
    objective(n_raters, seed)

## Variance based Search

In [ ]:
sim_name = sim_prefix+"OA045"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.045

for n_raters in [80, 110, 140, 170, 200]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-02 18:50:00.710019

    p_diff=0.045, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_OA045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-02 23:18:55.316142

    p_diff=0.045, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_OA045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-03 03:57:32.076223

    p_diff=0.045, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_OA045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-03 08:49:28.551546

    p_diff=0.045, <class 'float'>
    n_raters=170, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_OA045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-03 13:48:35.088571

    p_diff=0.045, <class 'float'>
    n_raters=200, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_OA045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [ ]:
sim_name = sim_prefix+"OA055"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.055

for n_raters in [40, 60, 80, 110, 140]:
    print(datetime.now())
    objective(n_raters, seed)

In [ ]:
sim_name = sim_prefix+"OA070"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.070

for n_raters in [40, 50, 60, 70, 80]:
    print(datetime.now())
    objective(n_raters, seed)

## Neyman Algo

In [11]:
sim_name = sim_prefix+"NM045"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.045

for n_raters in [80, 110, 140, 170, 200]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-08 08:43:44.060025

    p_diff=0.045, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 09:21:00.599917

    p_diff=0.045, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 10:03:47.409958

    p_diff=0.045, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 10:55:04.466843

    p_diff=0.045, <class 'float'>
    n_raters=170, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 11:57:02.355604

    p_diff=0.045, <class 'float'>
    n_raters=200, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [12]:
sim_name = sim_prefix+"NM055"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.055

for n_raters in [40, 60, 80, 110, 140]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-08 13:08:10.916005

    p_diff=0.055, <class 'float'>
    n_raters=40, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 13:47:47.756441

    p_diff=0.055, <class 'float'>
    n_raters=60, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 14:31:05.285690

    p_diff=0.055, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 15:16:52.799154

    p_diff=0.055, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 16:08:30.055926

    p_diff=0.055, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [13]:
sim_name = sim_prefix+"NM070"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.070

for n_raters in [40, 50, 60, 70, 80]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-08 17:04:37.604066

    p_diff=0.07, <class 'float'>
    n_raters=40, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 17:47:24.844062

    p_diff=0.07, <class 'float'>
    n_raters=50, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 18:31:51.822404

    p_diff=0.07, <class 'float'>
    n_raters=60, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 19:17:18.913134

    p_diff=0.07, <class 'float'>
    n_raters=70, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 20:04:16.232392

    p_diff=0.07, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=400
    seed=42, <class 'int'>
    sim_name=power_3_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [14]:
# Simulation settings
sim_prefix = "power_4_"
N_PROCESSES = 6
trials_per_sim = 1100
n_runs=30
seed=442

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
# CI_LEVEL = 0.91

In [16]:
sim_name = sim_prefix+"NM045"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.045

for n_raters in [80, 110, 140, 170, 200]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-09 09:21:05.391761

    p_diff=0.045, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 12:05:03.805498

    p_diff=0.045, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 15:03:43.234456

    p_diff=0.045, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 18:20:33.677658

    p_diff=0.045, <class 'float'>
    n_raters=170, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 21:51:35.051874

    p_diff=0.045, <class 'float'>
    n_raters=200, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM045, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [15]:
sim_name = sim_prefix+"NM055"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.055

for n_raters in [40, 60, 80, 110, 140]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-08 20:52:53.410467

    p_diff=0.055, <class 'float'>
    n_raters=40, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-08 23:00:00.493851

    p_diff=0.055, <class 'float'>
    n_raters=60, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 01:16:28.083591

    p_diff=0.055, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 03:40:35.979892

    p_diff=0.055, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-09 06:20:14.857768

    p_diff=0.055, <class 'float'>
    n_raters=140, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [18]:
sim_name = sim_prefix+"NM070"
OA = True
sim_id = [0]

power_id, p_diff = 0, 0.070

for n_raters in [40, 50, 60, 70, 80]:
    print(datetime.now())
    objective(n_raters, seed)

2022-12-10 09:43:34.421569

    p_diff=0.07, <class 'float'>
    n_raters=40, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-10 12:50:05.167775

    p_diff=0.07, <class 'float'>
    n_raters=50, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-10 16:04:06.305387

    p_diff=0.07, <class 'float'>
    n_raters=60, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-10 19:23:07.135884

    p_diff=0.07, <class 'float'>
    n_raters=70, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

2022-12-10 22:50:28.475896

    p_diff=0.07, <class 'float'>
    n_raters=80, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=1100
    seed=42, <class 'int'>
    sim_name=power_4_NM070, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin